In [1]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0 = 2.0678e-15

In [2]:
e_eff=(11.45+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [3]:
import pandas as pd

# Q1 related parameters

In [4]:
def cap_func(Q1,Qg1,Qg2):
    return Q1+(1/Qg1+1/Qg2)**(-1)

In [5]:
def Ec(C):#fF
    return np.round((elc)**2/(2*C)/h/1e9*1e15,1) # return in MHz

In [6]:
# path=r'E:\Github2.0\QCreator\designs\ThreeQubitsPlusTwoQubits\maxwell_simulations'
path=r'C:\Python\Github\QCreator\designs\ThreeQubitsPlusTwoQubits'
# path=r'D:\NextCloud\Quantum Computing\ChipDesigns\Ivan_Gerhard_designs\Two_qubits_plus_coupler\Hamiltonian_creation'

In [7]:
data=pd.read_csv(path+'\Design_single_qubit_2.txt',sep='\t')# matrix should be done on pF

In [8]:
data

,Charge,ground,Q1_bottom,Q1_top,Resonator
Charge,0.007950,-0.007922,-0.000012,-0.000015,-0.000001
ground,-0.007922,0.319100,-0.152540,-0.079101,-0.079533
Q1_bottom,-0.000012,-0.152540,0.188610,-0.035626,-0.000435
Q1_top,-0.000015,-0.079101,-0.035626,0.118420,-0.003679
Resonator,-0.000001,-0.079533,-0.000435,-0.003679,0.083648


In [9]:
q_name='Q1_'
Q1_cap_ground_top=data[q_name+'top'][3]+data[q_name+'top'][2]
Q1_cap_ground_bottom=data[q_name+'bottom'][2]+data[q_name+'bottom'][3]
Q1_cap_top_bottom=-data[q_name+'bottom'][3]
Q1_cap=cap_func(Q1_cap_top_bottom,Q1_cap_ground_top,Q1_cap_ground_bottom)

In [10]:
Q1_cap_top_bottom,Q1_cap_ground_bottom,Q1_cap_ground_top

(0.035626, 0.152984, 0.082794)

In [11]:
Q1_cap

0.08934669190509717

In [12]:
Ec(Q1_cap)

216.8

In [13]:
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):#nA
    return I*F0/(2*np.pi)/1e9/h/1e9
def IEj(Ej):
    return Ej/(F0/(Q2*np.pi)/1e9/h/1e9)
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec
def Ec(C):#fF
    return elc**2/(2*C)/h/1e9*1e15

In [14]:
a1    = 0.300
a2    = 0.300 # Junction width in um
#Calculate Qubit frequency:
JJ_area = a1*a2*1e6 #in nm^2
print('Critical current density is: ',j*1e3, 'uA/um^2')
Ic = JJ_area*j #in nA
print('Critical current: ',Ic ,' nA')
print('Ej: ',Eji(Ic) ,' GHz')

print('junction area  in um: ',JJ_area*1e-6)

Critical current density is:  0.5 uA/um^2
Critical current:  45.0  nA
Ej:  22.35039212442379  GHz
junction area  in um:  0.09


In [15]:
f(Eji(Ic),Ec(Q1_cap)/1e3)

6.009296014275234

In [16]:
Eji(Ic)/(Ec(Q1_cap)/1e3)

103.0929250051171

### Resonator coupling

In [17]:
def line_qubit_coupling(Q1g,Q2g,Q1gc,Q2gc):
    return np.abs((Q1g*Q2gc-Q2g*Q1gc)/(Q1g+Q2gc+Q2g+Q1gc))

In [18]:
def cap_pretty_print(args,coeff):
    for i in args:
        print(np.round(i*coeff,1))# return in fF

In [19]:
Q1_cap_res_top=-data[q_name+'top'][-1]
Q1_cap_res_bottom=-data[q_name+'bottom'][-1]
Ccr=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_res_top,Q1_cap_res_bottom)

In [20]:
cap_pretty_print([Q1_cap_res_top,Q1_cap_res_bottom,Ccr],1e3)

3.7
0.4
2.2


In [21]:
def qubit_res_coupling(Cc,Cq,wq,wr,Zr):# capacitances in fF, frequencies in GHz
    g= Cc*10**(-12)/2*np.sqrt((2*np.pi*wq*10**(9)*Zr/(Cq*10**(-12))))*wr*10**(9)
    return np.round(g/1e6,1)# return g/2pi in MHz

In [22]:
Zr=52.6
omega_r=5.6
omega_q=4.8
gqr=qubit_res_coupling(Ccr,Q1_cap,omega_q,omega_r,Zr)
gqr

25.9

### Dispersive shift

In [23]:
def dispersive_shift(gqr,wq,wr,Ec): # gqr in MHz,wq,wr in GHZ, EC in MHz and positive
    chi=gqr**2/((wq-wr)*10**(3))/(1+(wq-wr)*10**(3)/(-Ec))
    return chi # in MHz

In [24]:
chi=dispersive_shift(gqr,omega_q,omega_r,Ec(Q1_cap))
chi

-0.17878493744447566

### Readout resonator properties

In [30]:
Q=20000
k=2*np.pi*omega_r*1e9/Q 
np.round(1/k*1e9,1), np.round(k/Mhz,2)# first value in ns

(568.4, 0.28)

In [31]:
Tp=(1/k)*((omega_q-omega_r)*10**3/gqr)**2
Tp*1e6#us

86.63473724165266

### Charge line

In [59]:
Q1_cap_charge_line_top=-data[q_name+'top'][0]
Q1_cap_charge_line_bottom=-data[q_name+'bottom'][0]
Cc_charge=line_qubit_coupling(Q1_cap_ground_top,Q1_cap_ground_bottom,Q1_cap_charge_line_top,Q1_cap_charge_line_bottom)
cap_pretty_print([Q1_cap_charge_line_top,Q1_cap_charge_line_bottom,Cc_charge],1e6)#in aF

76.6
93.6
10.5


In [60]:
Cc_charge*1e6

10.545244379730185

In [62]:
omega_q=4.2

In [63]:
Z0=50
Tcharge=(Cc_charge+Q1_cap)/(Z0*(2*np.pi*omega_q*Cc_charge)**2)*1e-6

In [64]:
Tcharge*1e6#in us

25026.113067607275

### Total relaxation= resonator+charge line

In [65]:
Tp*Tcharge/(Tp+Tcharge)*1e6

119.71086492265829

In [ ]:
Ls=0.14fH